# Credit Data Analysis

For this project, we will examine a dataset containing features about individuals who applied for a loan. This dataset will need to be cleaned and examined for issues. After cleaning, we will analyze it to find out if a customer’s marital status, number of children, income, and loan purpose have an impact on whether they will default on a loan.

This analysis will be considered when building a **credit score** of a potential customer. A **credit score** is used to evaluate the ability of a potential borrower to repay their loan.

## Initial Examination

In [1]:
import pandas as pd

In [2]:
credit_data = pd.read_csv('credit_scoring_eng.csv')
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
credit_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [4]:
display(credit_data.describe())

# let's also get a sense for how many missing values we may have (if any)
credit_data.isnull().sum()/len(credit_data) * 100

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64

### Conclusion

This dataset seems to have a few interesting characteristics we will need to address to properly analyze any findings:
* There are missing values we will need to examine (~10% of two columns)
* There is at least one instance of -1 children, so we will need to examine that as well.
* The "days_employed" column has some negative values we will need to adjust. Normally, this would be assessed with insight from the data collector or data collection strategy.
* "education" has some upper/lower case variability we will need to fix.
    * All of the object columns may have this issue so we will adjust all of them for good measure.
* "purpose" may also have some case variability as well as inconsistent descriptions for similar purposes (purchase of the house == purchase of the house for my family) so we will need to examine and fix those as well.
* The data types are what we would expect - a mix of int, float, and object types.

## Data preprocessing

### Processing missing values

In [5]:
credit_data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

As we saw in the df info - there are two columns with missing values. Let's examine these further to see why they are missing before we develop a strategy to deal with them. It is suspicious that the number of missing values in each column are the same. Let's hypothesize that if the individual is unemployed ("days_employed" == 0) then their total income would also be 0 or missing.

In [6]:
test_data = credit_data['days_employed'].notnull()

credit_data[test_data].isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

As the above shows, when we remove the rows where "days_employed" == 0, we also removed all of the rows where "total_income" == 0. This appears to confirm our hypothesis that the missing values are missing for a real reason, and not due to some data collection mistake. In this case, we will simply replace the null values with 0.

In [7]:
credit_data.fillna(0, inplace=True)

# check to make sure
credit_data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Conclusion

We determined a legitimate reason for the null values and replaced them with a reasonable substitute. If an individual is unemployed ("days_employed" == 0 or null) then it would make sense for the "total_income" to also be 0 or null. All null values are now replaced with 0. Let's dig into this dataframe even more and continue pre-processing.

### Data type replacement

In [8]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


There looks to be some more work ahead of us. "days_employed" would be better off as an integer value as we are not interested in breaking the days down by hour or minute. Due to the existance of some negative values in there, we will also need to convert them to positive. The negative values are nonsensical for this column and may be due to an issue during data collection. Since we cannot communicate directly with the data collector or see the data collection process first hand, this is the conclusion we will sit with. In a real-world scenario, it would be preferable to ask for some insight before switching the sign of these values. We can also round "total_income" to an integer as the decimals after are not crucial for our analysis.<br><br>
From the "describe" function, we saw that the minimum and maximum values of the "debt" column are 0 and 1. We can convert this into a boolean to more easily understand the results of our analysis.
<br><br>
To wrap everything up, we will enforce a lower case on all of the strings in our categorical columns. By having all of these set to the same case, we will be able to group and compare them more easily. 

In [9]:
credit_data['days_employed'] = credit_data['days_employed'].astype(int)
credit_data['days_employed'] = abs(credit_data['days_employed']) # to remove the negative values
credit_data['total_income'] = credit_data['total_income'].astype(int)
credit_data['debt'] = credit_data['debt'].astype(bool)
credit_data['education'] = credit_data['education'].str.lower()
credit_data['family_status'] = credit_data['family_status'].str.lower()
credit_data['income_type'] = credit_data['income_type'].str.lower()
credit_data['purpose'] = credit_data['purpose'].str.lower()

# confirm
credit_data.info()
credit_data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  bool  
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: bool(1), int32(2), int64(4), object(5)
memory usage: 1.7+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,bachelor's degree,0,married,0,F,employee,False,40620,purchase of the house
1,1,4024,36,secondary education,1,married,0,F,employee,False,17932,car purchase
2,0,5623,33,secondary education,1,married,0,M,employee,False,23341,purchase of the house
3,3,4124,32,secondary education,1,married,0,M,employee,False,42820,supplementary education
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,False,25378,to have a wedding
5,0,926,27,bachelor's degree,0,civil partnership,1,M,business,False,40922,purchase of the house
6,0,2879,43,bachelor's degree,0,married,0,F,business,False,38484,housing transactions
7,0,152,50,secondary education,1,married,0,M,employee,False,21731,education
8,2,6929,35,bachelor's degree,0,civil partnership,1,F,employee,False,15337,having a wedding
9,0,2188,41,secondary education,1,married,0,M,employee,False,23108,purchase of the house for my family


In [10]:
credit_data_proc = credit_data[[
    'children', 'days_employed', 'dob_years', 'education', 'family_status', 'gender', 'income_type', 'debt', 'total_income', 'purpose'
]]

credit_data_proc.head(10)

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose
0,1,8437,42,bachelor's degree,married,F,employee,False,40620,purchase of the house
1,1,4024,36,secondary education,married,F,employee,False,17932,car purchase
2,0,5623,33,secondary education,married,M,employee,False,23341,purchase of the house
3,3,4124,32,secondary education,married,M,employee,False,42820,supplementary education
4,0,340266,53,secondary education,civil partnership,F,retiree,False,25378,to have a wedding
5,0,926,27,bachelor's degree,civil partnership,M,business,False,40922,purchase of the house
6,0,2879,43,bachelor's degree,married,F,business,False,38484,housing transactions
7,0,152,50,secondary education,married,M,employee,False,21731,education
8,2,6929,35,bachelor's degree,civil partnership,F,employee,False,15337,having a wedding
9,0,2188,41,secondary education,married,M,employee,False,23108,purchase of the house for my family


### Conclusion

We have now removed all of the missing values and adjusted our data/data types to make it easier to analyze our dataframe. Our values are meaningful and our string cases are consistent. There is still some more work to be done but we are closer to making a true analysis. 

### Processing duplicates

In [11]:
credit_data_duplicates = credit_data_proc[credit_data_proc.duplicated(keep=False)]

display(credit_data_duplicates)
credit_data_duplicates.shape

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose
120,0,0,46,secondary education,married,F,employee,False,0,university education
520,0,0,35,secondary education,civil partnership,F,employee,False,0,to have a wedding
541,0,0,57,secondary education,married,F,employee,False,0,second-hand car purchase
554,0,0,60,secondary education,married,M,employee,False,0,buy real estate
680,1,0,30,bachelor's degree,married,F,civil servant,False,0,purchase of the house for my family
...,...,...,...,...,...,...,...,...,...,...
20702,0,0,64,secondary education,married,F,retiree,False,0,supplementary education
21032,0,0,60,secondary education,married,F,retiree,False,0,to become educated
21132,0,0,47,secondary education,married,F,employee,False,0,housing renovation
21281,1,0,30,bachelor's degree,married,F,employee,False,0,buy commercial real estate


(137, 10)

We do not have a lot of duplicate data in this dataframe. Our analysis will not be hindered by the loss of less than 1% of our data. We will run the "drop duplicates" function to clean up any duplicated data. But before we do this, we should work on our "purpose" column some more. It's very likely that some of the purposes are similar and just written differently. We should break these down using the nltk library to categorize and assess for duplicates.

In [12]:
# Lets first see what unique values we are dealing with
def print_uniques(df, string_cols):
    """
    Print the unique values for a specified columns
    """
    for column in string_cols:
        print("The number of unique values in", column, "are:")
        print(df[column].value_counts())
        print()
        print()
    
# Now to run our function over the columns we are interested in
credit_data_str = credit_data_proc[['education', 'family_status', 'gender', 'income_type', 'purpose']]
string_cols = ['education', 'family_status', 'gender', 'income_type', 'purpose']

print_uniques(credit_data_str, string_cols)

The number of unique values in education are:
secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64


The number of unique values in family_status are:
married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64


The number of unique values in gender are:
F      14236
M       7288
XNA        1
Name: gender, dtype: int64


The number of unique values in income_type are:
employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64


The number of unique values in purpose are:
wedding ceremony       

All of the columns seem to have reasonably different values (taking note of the "XNA" value in "gender") with the exception of "purpose". The "purpose" column has many values that are similar. It would be best to break these into their stems and organize them in that manner.

In [13]:
from nltk.stem import SnowballStemmer 
english_stemmer = SnowballStemmer('english')

# first we need to split up the strings in the "purpose" column
credit_data_proc['purpose'] = credit_data_proc['purpose'].str.split()

# then we can run the stemmer
credit_data_proc['purpose_stems'] = credit_data_proc['purpose'].apply(lambda x:[english_stemmer.stem(word) for word in x])

# and drop the old purpose column
# now we have something we can analyze
credit_data_stemmed = credit_data_proc.drop(columns=["purpose"])
credit_data_stemmed.head(10)

C:\Users\kohar\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\kohar\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose_stems
0,1,8437,42,bachelor's degree,married,F,employee,False,40620,"[purchas, of, the, hous]"
1,1,4024,36,secondary education,married,F,employee,False,17932,"[car, purchas]"
2,0,5623,33,secondary education,married,M,employee,False,23341,"[purchas, of, the, hous]"
3,3,4124,32,secondary education,married,M,employee,False,42820,"[supplementari, educ]"
4,0,340266,53,secondary education,civil partnership,F,retiree,False,25378,"[to, have, a, wed]"
5,0,926,27,bachelor's degree,civil partnership,M,business,False,40922,"[purchas, of, the, hous]"
6,0,2879,43,bachelor's degree,married,F,business,False,38484,"[hous, transact]"
7,0,152,50,secondary education,married,M,employee,False,21731,[educ]
8,2,6929,35,bachelor's degree,civil partnership,F,employee,False,15337,"[have, a, wed]"
9,0,2188,41,secondary education,married,M,employee,False,23108,"[purchas, of, the, hous, for, my, famili]"


In [14]:
# lets see how many unique values we have now
credit_data_stemmed['purpose_stems'].value_counts()

[car]                                        973
[wed, ceremoni]                              797
[have, a, wed]                               777
[to, have, a, wed]                           774
[real, estat, transact]                      676
[buy, commerci, real, estat]                 664
[hous, transact]                             653
[buy, properti, for, rent, out]              653
[transact, with, commerci, real, estat]      651
[purchas, of, the, hous]                     647
[hous]                                       647
[purchas, of, the, hous, for, my, famili]    641
[construct, of, own, properti]               635
[properti]                                   634
[transact, with, my, real, estat]            630
[build, a, real, estat]                      626
[buy, real, estat]                           624
[build, a, properti]                         620
[purchas, of, my, own, hous]                 620
[hous, renov]                                612
[buy, residenti, rea

Now that we can see our stems, we can create a dictionary and write a function to parse the stems and convert them into a general purpose category. The main purpose reasons seem to revolve around "car", "wedding", "real estate", "house", and "education".

In [15]:
cat_dict = {
    'car':'car',
    'wed':'wedding',
    'estat':'real estate',
    'hous':'house',
    'properti':'house',
    'univers':'education',
    'educ':'education'
}

def categorize_purpose(word):
    """
    Categorize the purpose column by examining the stem.
    """
    for key in cat_dict.keys():
        if key in word:
            return cat_dict.get(key)
    return None

credit_data_stemmed['purpose_cat'] = credit_data_stemmed['purpose_stems'].apply(lambda x: categorize_purpose(x))
credit_data_stemmed.head(10)

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose_stems,purpose_cat
0,1,8437,42,bachelor's degree,married,F,employee,False,40620,"[purchas, of, the, hous]",house
1,1,4024,36,secondary education,married,F,employee,False,17932,"[car, purchas]",car
2,0,5623,33,secondary education,married,M,employee,False,23341,"[purchas, of, the, hous]",house
3,3,4124,32,secondary education,married,M,employee,False,42820,"[supplementari, educ]",education
4,0,340266,53,secondary education,civil partnership,F,retiree,False,25378,"[to, have, a, wed]",wedding
5,0,926,27,bachelor's degree,civil partnership,M,business,False,40922,"[purchas, of, the, hous]",house
6,0,2879,43,bachelor's degree,married,F,business,False,38484,"[hous, transact]",house
7,0,152,50,secondary education,married,M,employee,False,21731,[educ],education
8,2,6929,35,bachelor's degree,civil partnership,F,employee,False,15337,"[have, a, wed]",wedding
9,0,2188,41,secondary education,married,M,employee,False,23108,"[purchas, of, the, hous, for, my, famili]",house


In [16]:
# we'll drop the "stems" column now that it has served it's purpose
credit_data_stemmed = credit_data_stemmed.drop(columns=["purpose_stems"])

# confirm
credit_data_stemmed.head(10)

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose_cat
0,1,8437,42,bachelor's degree,married,F,employee,False,40620,house
1,1,4024,36,secondary education,married,F,employee,False,17932,car
2,0,5623,33,secondary education,married,M,employee,False,23341,house
3,3,4124,32,secondary education,married,M,employee,False,42820,education
4,0,340266,53,secondary education,civil partnership,F,retiree,False,25378,wedding
5,0,926,27,bachelor's degree,civil partnership,M,business,False,40922,house
6,0,2879,43,bachelor's degree,married,F,business,False,38484,house
7,0,152,50,secondary education,married,M,employee,False,21731,education
8,2,6929,35,bachelor's degree,civil partnership,F,employee,False,15337,wedding
9,0,2188,41,secondary education,married,M,employee,False,23108,house


In [17]:
# now that we've cleaned our purposes, let's recheck the number of duplicates 
credit_data_duplicates = credit_data_stemmed[credit_data_stemmed.duplicated(keep=False)]

display(credit_data_duplicates)
credit_data_duplicates.shape

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose_cat
29,0,0,63,secondary education,unmarried,F,retiree,False,0,real estate
41,0,0,50,secondary education,married,F,civil servant,False,0,car
67,0,0,52,bachelor's degree,married,F,retiree,False,0,house
90,2,0,35,bachelor's degree,married,F,employee,False,0,house
97,0,0,47,bachelor's degree,married,F,employee,False,0,education
...,...,...,...,...,...,...,...,...,...,...
21321,0,0,56,secondary education,married,F,retiree,False,0,real estate
21391,0,0,52,secondary education,married,F,business,False,0,house
21414,0,0,65,secondary education,married,F,retiree,False,0,house
21415,0,0,54,secondary education,married,F,retiree,False,0,house


(553, 10)

The number of duplicated rows is much higher now that we've cleaned our purposes column. Even with the higher number of rows, we're still only considering ~2.5% of our data. Our analysis will still not be hindered by losing this small amount - so let's run the "drop duplicates" function to clean these up.

In [18]:
credit_data_stemmed.drop_duplicates()

# let's see how many rows were fully duplicated
credit_data_stemmed.shape

(21525, 10)

Overall nothing was lost by our "drop duplicates" function.

### Conclusion

Our "purpose" column has been broken down and cleaned into a much more helpful column - describing the purpose in a single word that we can categorize and analyze our data with. Overall this led to no fully duplicated rows. In the process of checking for duplicates, we were also able to categorize our data by purpose. There are some odd rows and values noted that we will examine in the next section. After these are looked at, we will categorize our data and start to examine any relationships.

### Other problematic values

In [19]:
# first, let's look at the "XNA" gender row
credit_data_stemmed.loc[credit_data_stemmed['gender'] == "XNA"]

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose_cat
10701,0,2358,24,some college,civil partnership,XNA,business,False,32624,real estate


Most likely, XNA is an intersex or unspecified value. Since it's the only value in our dataframe we can either ignore it or remove it in our dataset. It won't hurt our analysis so we will keep it in for now. 

In [20]:
# next we'll look at the -1 children row(s)
neg_child = credit_data_stemmed.loc[credit_data_stemmed['children'] == -1]
display(neg_child)
neg_child.shape

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose_cat
291,-1,4417,46,secondary education,civil partnership,F,employee,False,16450,education
705,-1,902,50,secondary education,married,F,civil servant,False,22061,car
742,-1,3174,57,secondary education,married,F,employee,False,10282,education
800,-1,349987,54,secondary education,unmarried,F,retiree,False,13806,education
941,-1,0,57,secondary education,married,F,retiree,False,0,car
1363,-1,1195,55,secondary education,married,F,business,False,11128,education
1929,-1,1461,38,secondary education,unmarried,M,employee,False,17459,house
2073,-1,2539,42,secondary education,divorced,F,business,False,26022,house
3814,-1,3045,26,secondary education,civil partnership,F,civil servant,False,21102,wedding
4201,-1,901,41,secondary education,married,F,civil servant,False,36220,real estate


(47, 10)

There doesn't seem to be any clear pattern here for the 47 rows with -1 children.<br><br>
There is enough data here where it warrants removing it and not enough where that would pose any real risk of affecting our analysis (less than .3% of our data). 

In [21]:
credit_data_stemmed = credit_data_stemmed[credit_data_stemmed.children != -1]

# confirm
credit_data_stemmed.describe()

,children,days_employed,dob_years,total_income
count,21478.000000,21478.000000,21478.000000,21478.000000
mean,0.542276,60166.585390,43.294953,24083.873824
std,1.381219,133369.133879,12.577312,17593.360933
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,609.000000,33.000000,14176.250000
50%,0.000000,1807.000000,42.000000,21676.500000
75%,1.000000,4780.500000,53.000000,31298.500000
max,20.000000,401755.000000,75.000000,362496.000000


### Conclusion

The dataframe problem values have been examined and we are ready to continue categorizing so as to answer our initial questions.

## Analysis

- Is there a relation between having kids and repaying a loan on time?

In [22]:
credit_data_stemmed.head(10)

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose_cat
0,1,8437,42,bachelor's degree,married,F,employee,False,40620,house
1,1,4024,36,secondary education,married,F,employee,False,17932,car
2,0,5623,33,secondary education,married,M,employee,False,23341,house
3,3,4124,32,secondary education,married,M,employee,False,42820,education
4,0,340266,53,secondary education,civil partnership,F,retiree,False,25378,wedding
5,0,926,27,bachelor's degree,civil partnership,M,business,False,40922,house
6,0,2879,43,bachelor's degree,married,F,business,False,38484,house
7,0,152,50,secondary education,married,M,employee,False,21731,education
8,2,6929,35,bachelor's degree,civil partnership,F,employee,False,15337,wedding
9,0,2188,41,secondary education,married,M,employee,False,23108,house


In [23]:
# first, we need to create a function to categorize our rows by whether or not they have kids

def has_kids(children):
    """
    Returns a string dependent on "children" column value
    """
    if children > 0:
        return 'yes'
    if children == 0:
        return 'no'
    
# next we'll apply it to our df
credit_data_stemmed['has_kids'] = credit_data_stemmed['children'].apply(has_kids)

# now we can use this to see if there is a relationship
q1_df = credit_data_stemmed.groupby('has_kids')['debt'].value_counts()
q1_df

has_kids  debt 
no        False    13086
          True      1063
yes       False     6652
          True       677
Name: debt, dtype: int64

In [24]:
no_kid = q1_df.loc[('no', True)] / (q1_df.loc[('no', True)] + q1_df.loc[('no', False)])
kid = q1_df.loc[('yes', True)] / (q1_df.loc[('yes', True)] + q1_df.loc[('yes', False)])

print("Individuals with no kids are in debt {:.2%} of the time".format(no_kid))
print("Individuals with kids are in debt {:.2%} of the time".format(kid))

Individuals with no kids are in debt 7.51% of the time
Individuals with kids are in debt 9.24% of the time


### Conclusion

There appears to be a slight increase in one's chance to be in debt if they have kids. 

- Is there a relation between marital status and repaying a loan on time?

In [25]:
# we can do this with the dataframe as is using groupby
q2_df = credit_data_stemmed.groupby('family_status')['debt'].value_counts()
q2_df

family_status      debt 
civil partnership  False     3784
                   True       388
divorced           False     1106
                   True        85
married            False    11421
                   True       930
unmarried          False     2534
                   True       274
widow / widower    False      893
                   True        63
Name: debt, dtype: int64

In [26]:
civil = q2_df.loc[('civil partnership', True)] \
/ (q2_df.loc[('civil partnership', True)] + q2_df.loc[('civil partnership', False)])

divorced = q2_df.loc[('divorced', True)] / (q2_df.loc[('divorced', True)] + q2_df.loc[('divorced', False)])
married = q2_df.loc[('married', True)] / (q2_df.loc[('married', True)] + q2_df.loc[('married', False)])
unmarried = q2_df.loc[('unmarried', True)] / (q2_df.loc[('unmarried', True)] + q2_df.loc[('unmarried', False)])
widow = q2_df.loc[('widow / widower', True)] \
/ (q2_df.loc[('widow / widower', True)] + q2_df.loc[('widow / widower', False)])

print("Civil partnerships are in debt {:.2%} of the time".format(civil))
print("Divorced are in debt {:.2%} of the time".format(divorced))
print("Married are in debt {:.2%} of the time".format(married))
print("Unmarried are in debt {:.2%} of the time".format(unmarried))
print("Widows/widowers are in debt {:.2%} of the time".format(widow))

Civil partnerships are in debt 9.30% of the time
Divorced are in debt 7.14% of the time
Married are in debt 7.53% of the time
Unmarried are in debt 9.76% of the time
Widows/widowers are in debt 6.59% of the time


### Conclusion

Civil partnerships and unmarried individuals are more likely to be in debt. Widow/widowers are the least likely, with divorced and married individuals not far behind.

- Is there a relation between income level and repaying a loan on time?

In [27]:
# first, let's see what the income ranges are and then we'll group them accordingly
credit_data_stemmed.describe()

,children,days_employed,dob_years,total_income
count,21478.000000,21478.000000,21478.000000,21478.000000
mean,0.542276,60166.585390,43.294953,24083.873824
std,1.381219,133369.133879,12.577312,17593.360933
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,609.000000,33.000000,14176.250000
50%,0.000000,1807.000000,42.000000,21676.500000
75%,1.000000,4780.500000,53.000000,31298.500000
max,20.000000,401755.000000,75.000000,362496.000000


In [28]:
# If we take it by the percentages, we can break it into three catagories to see if there is any relationship

def income_category(income):
    """
    Returns a string dependent on "total_income" column value1
    """
    if income < 15000:
        return "low"
    if 15000 <= income < 35000:
        return "medium"
    if income >= 35000:
        return "high"
    
# now to generate our column
credit_data_stemmed['income_cat'] = credit_data_stemmed['total_income'].apply(income_category)

# and lastly, we'll check the relationship
q3_df = credit_data_stemmed.groupby('income_cat')['debt'].value_counts()
q3_df

income_cat  debt 
high        False     3771
            True       290
low         False     5439
            True       468
medium      False    10528
            True       982
Name: debt, dtype: int64

In [29]:
low = q3_df.loc[('low', True)] / (q3_df.loc[('low', True)] + q3_df.loc[('low', False)])
medium = q3_df.loc[('medium', True)] / (q3_df.loc[('medium', True)] + q3_df.loc[('medium', False)])
high = q3_df.loc[('high', True)] / (q3_df.loc[('high', True)] + q3_df.loc[('high', False)])

print("Low income individuals are in debt {:.2%} of the time".format(low))
print("Medium income individuals are in debt {:.2%} of the time".format(medium))
print("High income individuals are in debt {:.2%} of the time".format(high))

Low income individuals are in debt 7.92% of the time
Medium income individuals are in debt 8.53% of the time
High income individuals are in debt 7.14% of the time


### Conclusion

There does appear to be some relationship between income level and repaying a loan on time. Roughly 7% of individuals in the high income category are in debt, whereas about 8% of individuals in the low and medium income categories are in debt.

- How do different loan purposes affect on-time repayment of the loan?

In [30]:
# we can also do this analysis without any new functions
q4_df = credit_data_stemmed.groupby('purpose_cat')['debt'].value_counts()
q4_df

purpose_cat  debt 
car          False    3902
             True      402
education    False    3642
             True      370
house        False    5906
             True      446
real estate  False    4128
             True      336
wedding      False    2160
             True      186
Name: debt, dtype: int64

In [31]:
car = q4_df.loc[('car', True)] / (q4_df.loc[('car', True)] + q4_df.loc[('car', False)])
education = q4_df.loc[('education', True)] / (q4_df.loc[('education', True)] + q4_df.loc[('education', False)])
house = q4_df.loc[('house', True)] / (q4_df.loc[('house', True)] + q4_df.loc[('house', False)])
real_estate = q4_df.loc[('real estate', True)] / (q4_df.loc[('real estate', True)] + q4_df.loc[('real estate', False)])
wedding = q4_df.loc[('wedding', True)] / (q4_df.loc[('wedding', True)] + q4_df.loc[('wedding', False)])

print("Car loans are in debt {:.2%} of the time".format(car))
print("Education loans are in debt {:.2%} of the time".format(education))
print("Housing loans are in debt {:.2%} of the time".format(house))
print("Real estate loans are in debt {:.2%} of the time".format(real_estate))
print("Wedding loans are in debt {:.2%} of the time".format(wedding))

Car loans are in debt 9.34% of the time
Education loans are in debt 9.22% of the time
Housing loans are in debt 7.02% of the time
Real estate loans are in debt 7.53% of the time
Wedding loans are in debt 7.93% of the time


### Conclusion

From the above table, we can see that loans for housing, wedding, and real estate are less likely to result in failing to repay on time - so these would be the safest to offer loans for. Car and education loans are the most likely to result in debt, and should therefore be considered a higher risk. 

## General conclusion

This dataset had a few issues we needed to address before we could analyze it properly. We started by exploring the missing values and noticed that there was a relationship between the nulls. After this relationship was found, we developed a reasonable solution to fill them. Next we adjusted our data types in each column to make the data easier to group and examine for duplicates. To fully search for duplicates, we needed to clean the "purpose" column and trim them down to five purpose categories. After cleaning that column, we found no fully duplicated rows. Lastly, we took a closer look at a couple of problems we noticed in our earlier examination. Once those issues were addressed, we were ready to start our analysis. 

In our effort to determine if a customer’s marital status, number of children, income, and loan purpose have an impact on whether they will default on a loan, we found some interesting relationships.

From this dataset, we can see that the highest risk loans are for individuals that:
* Have kids (9.24%)
* Are in civil partnerships or unmarried (9.30% and 9.76%, respectively)
* Have a lower income (~8% for low/middle income vs. ~7% for high)
* Are applying for a car or education loan (9.34% and 9.22%, respectively)